[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_convnet.ipynb)

In [0]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='nadam',
            loss='categorical_crossentropy',
            metrics=[ 'accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  
  h = [1., 0.5, 0.3, -0.13, 0.2, 0, 0.4]
  #h = np.random.randn(1,chanelLen)
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h[0:chanelLen],mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh ,y ]
  
                        
data={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)

In [37]:
# Visulizing the data

m = 16         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 50000   # number of trials 
Noise = 1
SNRdB = 20
chanelLen = 2


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  
print(seqLen)

generateData2(m,seqLen,trials,Noise,SNRdB,chanelLen)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
#print(data['x_train']) # so u can see the data 

# tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

# #plot 1
# plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
#                                          # I only plot the first 10 points from 
#                                          # the sequence because as time goes on 
#                                          # the data becomes less dependent on the
#                                          # previous symbols. Even 10 is a lot. 
# plt.title('4-QAM Mod With Multi-Channel AWGN plot 1')
# plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



256
(50000, 257, 2)
(50000, 256)


In [38]:

# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(data['y_train'] , num_classes=m)
y_val = keras.utils.to_categorical(data['y_val'] , num_classes=m)
y_test = keras.utils.to_categorical(data['y_test'] , num_classes=m)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}


(50000, 257, 2)
(50000, 257, 2)
(50000, 256, 16)
(25000, 257, 2)


In [31]:
# QAM-4
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(2,kernel_size=(3), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(2,kernel_size=(3)))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=3)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_35 (Conv1D)           (None, 259, 2)            14        
_________________________________________________________________
batch_normalization_52 (Batc (None, 259, 2)            8         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 257, 2)            14        
_________________________________________________________________
batch_normalization_53 (Batc (None, 257, 2)            8         
_________________________________________________________________
flatten_18 (Flatten)         (None, 514)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 4096)              2109440   
_________________________________________________________________
reshape_18 (Reshape)         (None, 256, 16)           0         
__________

 3520/10000 [=========>....................] - ETA: 2s - loss: 2.2021 - acc: 0.2865

KeyboardInterrupt: ignored

In [0]:
# QAM-16
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(90,kernel_size=(10), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(90,kernel_size=(10)))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_43 (Conv1D)           (None, 248, 90)           1890      
_________________________________________________________________
batch_normalization_64 (Batc (None, 248, 90)           360       
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 239, 90)           81090     
_________________________________________________________________
batch_normalization_65 (Batc (None, 239, 90)           360       
_________________________________________________________________
flatten_22 (Flatten)         (None, 21510)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 4096)              88109056  
_________________________________________________________________
reshape_22 (Reshape)         (None, 256, 16)           0         
__________

50000/50000 [==============================] - 137s 3ms/step - loss: 1.8606 - acc: 0.3693 - val_loss: 1.4891 - val_acc: 0.4446
Epoch 2/50
11072/50000 [=====>........................] - ETA: 1:39 - loss: 1.4602 - acc: 0.5140

50000/50000 [==============================] - 133s 3ms/step - loss: 1.3676 - acc: 0.5440 - val_loss: 1.2123 - val_acc: 0.5705
Epoch 3/50
13888/50000 [=======>......................] - ETA: 1:31 - loss: 1.1789 - acc: 0.6251

50000/50000 [==============================] - 133s 3ms/step - loss: 1.1242 - acc: 0.6458 - val_loss: 1.0167 - val_acc: 0.6673
Epoch 4/50
14976/50000 [=======>......................] - ETA: 1:29 - loss: 0.9861 - acc: 0.7047

50000/50000 [==============================] - 133s 3ms/step - loss: 0.9451 - acc: 0.7221 - val_loss: 0.8613 - val_acc: 0.7414
Epoch 5/50
15360/50000 [========>.....................] - ETA: 1:28 - loss: 0.8337 - acc: 0.7677

...

50000/50000 [==============================] - 133s 3ms/step - loss: 0.0933 - acc: 0.9845 - val_loss: 0.0908 - val_acc: 0.9841
Epoch 22/50
15552/50000 [========>.....................] - ETA: 1:27 - loss: 0.0872 - acc: 0.9848

50000/50000 [==============================] - 133s 3ms/step - loss: 0.0875 - acc: 0.9843 - val_loss: 0.0863 - val_acc: 0.9835
Epoch 23/50
15552/50000 [========>.....................] - ETA: 1:27 - loss: 0.0816 - acc: 0.9849

50000/50000 [==============================] - 133s 3ms/step - loss: 0.0806 - acc: 0.9851 - val_loss: 0.0790 - val_acc: 0.9847
Epoch 24/50
15552/50000 [========>.....................] - ETA: 1:27 - loss: 0.0784 - acc: 0.9844

50000/50000 [==============================] - 133s 3ms/step - loss: 0.0782 - acc: 0.9844 - val_loss: 0.0770 - val_acc: 0.9844
Epoch 25/50
15552/50000 [========>.....................] - ETA: 1:27 - loss: 0.0761 - acc: 0.9840

37184/50000 [=====================>........] - ETA: 32s - loss: 0.0755 - acc: 0.9843